In [1]:
import pandas as pd
import requests 
from github import Github
from github import RateLimitExceededException
from github import UnknownObjectException
from datetime import datetime, timedelta
import pickle
import time
import itertools
import os
import calendar

In [21]:
user_roles = pd.read_csv('..\dev_name_users.csv')

## Collect Repository dataset

In [42]:
class GitHubScrapper:

    def __init__(self):
        auth_git = #put your github authentication key here
        self.g = Github(auth_git)

    def user_info_extracter(self, users):

        #users_iter = iter(users)

        for u in users:
            try:
                user = self.g.get_user(u)
                try:
                    name = user.name

                except:
                    name = "none"
                    
                try:
                    u_id = user.login

                except:
                    u_id = "none"

                try:
                    date = user.created_at
                except:
                    date = "none"


                try:
                    activity = user.last_modified
                except:
                    activity = "none"

                try:
                    html_url = user.html_url
                except:
                    html_url = "none"

                try:
                    email = user.email
                except:
                    email = "none"

                try:
                    location = user.location
                except:
                    location = "none"

                try:
                    blog = user.blog
                except:
                    blog = "none"

                try:
                    bio = user.bio
                except:
                    bio = "none"

                try:
                    reps = user.get_repos()
                except:
                    reps = "none"

                full_rep = {}
                cnt = 0
                for repository in reps:
                    repo = {}
                    try:
                        repo_name = repository.name
                    except:
                        repo_name = "none"

                    repo['repo_name'] = repo_name

                    try:
                        readme = repository.get_contents('README.md').decoded_content
                    except:
                        readme = "none"

                    repo['readme'] = readme
                    
                    try:
                        repo_topic = repository.topics
                    except:
                        repo_topic = "none"

                    repo['topics'] = repo_topic
                    
                    
                    try:
                        repo_about = repository.description
                    except:
                        repo_about = "none"

                    repo['about'] = repo_about

                    try:
                        stars = repository.stargazers_count
                    except:
                        stars = "none"

                    repo['stars'] = stars

                    try:
                        perc = 0
                        lang_perc = {}
                        for key, value in repository.get_languages().items():
                            perc += value

                        for key, value in repository.get_languages().items():
                            percentage = value / perc * 100
                            lang_perc[key] = percentage
                        repo['languages'] = lang_perc
                    except:
                        repo['languages'] = "none"


                    try:
                        total_commits = repository.get_commits().totalCount
                    except:
                        total_commits =1
            
                    try:
                        total_commits_author = repository.get_commits(author=user.login).totalCount
                    except:
                        total_commits_author=0
                        
                    try:    
                        author_contribution = total_commits_author / total_commits * 100
                    except:
                        author_contribution=0
                        
                    try:   
                        commit_files = [item.filename.split(".")[-1] for cm in repository.get_commits(author=user.login) for item in cm.files]
                    except:
                        commit_files = []

                    try:
                        extension_count = {}
                        for extension in commit_files:
                            if extension not in extension_count.keys():
                                extension_count[extension] = 1
                            else:
                                extension_count[extension]+=1

                    except:
                        extension_count = {}


                    repo['total_commits'] = total_commits
                    repo['author_commits'] = total_commits_author
                    repo['author_contribution'] = author_contribution

                    repo['files'] = extension_count
                    full_rep[cnt] = repo
                    cnt += 1


                data = {}

                data['name'] = name
                data['u_id'] = u_id
                data['date'] = date
                data['activity'] = activity
                data['url'] = html_url
                data['email'] = email
                data['location'] = location
                data['blog'] = blog
                data['bio'] = bio
                data['repository'] = full_rep

                    
        
                pickle.dump(data, open(('../' + u_id + '.pickle'), 'wb'))
            

                print(u_id, "-------------------> done")

            except RateLimitExceededException:
                now = datetime.now()
                current_time = now.strftime("%H:%M:%S")
                search_rate_limit = self.g.get_rate_limit().search
                print('current time:{}'.format(current_time))
                #time.sleep(sleep_time)
                time.sleep(3600)
                continue
            except UnknownObjectException:
                continue
                
    def get_user_details(self):
        self.user_info_extracter(user_roles)    

In [43]:
gg = GitHubScrapper()
gg.get_user_details()

pronix -------------------> done
